# LightFM - Гибридная рекомендательная система

Рассмотрим возможность добавления item_features и user_features в модель LightFM при помощи lightfm.dataset

Основной ноутбук: https://www.kaggle.com/fedorazarov/project-20-group-1-fa

# Подготовка данных

In [1]:
import numpy as np
import pandas as pd
from collections import Counter
import json
import re

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Загружаем датасеты
train = pd.read_csv('/kaggle/input/recommendationsv4/train.csv')
test = pd.read_csv('/kaggle/input/recommendationsv4/test.csv')
submission = pd.read_csv('/kaggle/input/recommendationsv4/sample_submission.csv')

# Постройчно прочитаем json с метаданными и положим результат в датасет "meta"
with open('/kaggle/input/recommendationsv4/meta_Grocery_and_Gourmet_Food.json') as f:
    meta_list = []
    for line in f.readlines():
        meta_list.append(json.loads(line))
        
meta = pd.DataFrame(meta_list)

# Удалим дубликаты из тренировочного датасета
train.drop_duplicates(inplace = True)

# Объединим тренировочный датасет и данные из meta по идентификатору asin (Amazon Standard Identification Number)
df = pd.merge(train, meta, on='asin')
#df_new_test = pd.merge(test, meta, on='asin')

/kaggle/input/recommendationsv4/meta_Grocery_and_Gourmet_Food.json
/kaggle/input/recommendationsv4/sample_submission.csv
/kaggle/input/recommendationsv4/test.csv
/kaggle/input/recommendationsv4/train.csv


/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Обработаем только по одному признаку для пользователей и продуктов для скорости работы ноутбука: verified и main_cat

In [2]:
dic_verified = {
    True: 1,
    False: 0
}
df['verified'] = df['verified'].map(dic_verified)

In [3]:
# Заменим пропуски в main_cat на категорию "Other"
df.main_cat = df.main_cat.fillna('Other')

In [4]:
df.columns

Index(['overall', 'verified', 'reviewTime', 'asin', 'reviewerName',
       'reviewText', 'summary', 'unixReviewTime', 'vote', 'style', 'image_x',
       'userid', 'itemid', 'rating', 'category', 'description', 'title',
       'brand', 'rank', 'also_view', 'main_cat', 'price', 'also_buy',
       'image_y', 'date', 'feature', 'details', 'similar_item', 'tech1',
       'fit'],
      dtype='object')

In [5]:
features_user = df[['userid', 'verified']]
features_item = df[['itemid', 'main_cat']]
df = df[['userid','itemid','rating']]

Нам нужно вызвать метод fit, чтобы сообщить LightFM id пользователей, id продуктов, и дополнительные фичи пользователя или продукта. 

Мы передадим методу fit три параметра:

    users: список всех пользователей
    items: список всех продуктов
    item_features: список дополнительных фичей продукта

Передача списка пользователей и продуктов довольно проста - просто используем столбцы «userid» и «itemid» из df.

Когда дело доходит до передачи item_features, я рекомендуется передать список, в котором каждый элемент имеет формат, подобный 'feature_name: feature_value'.

Это означает, что наши item_features должны выглядеть примерно так:
['feature1: 0', 'feature1: 1', 'feature2: 0', 'feature2: 1', 'feature3: 0', 'feature3: 1' ...].

Этот список должен быть создан с учетом всех возможных пар feature_name, feature_value, которые могут встретиться в обучающем датасете. Например, для feature_name, равного verified, может быть два feature_value, а именно 0 и 1.

Ниже небольшой фрагмент кода, который позволяет создать такой список (назовем его item_f):

In [6]:
item_f = []
col = []
unique_f1 = []
for column in features_item.drop(['itemid'], axis=1):
    col += [column]*len(features_item[column].unique())
    unique_f1 += list(features_item[column].unique())
for x,y in zip(col, unique_f1):
    res = str(x)+ ":" +str(y)
    item_f.append(res)
    print(res)

main_cat:Grocery
main_cat:Health & Personal Care
main_cat:Office Products
main_cat:Sports & Outdoors
main_cat:Amazon Home
main_cat:Toys & Games
main_cat:Other
main_cat:Industrial & Scientific
main_cat:All Beauty
main_cat:Tools & Home Improvement
main_cat:Baby
main_cat:Pet Supplies
main_cat:Home Audio & Theater
main_cat:Arts, Crafts & Sewing
main_cat:Camera & Photo
main_cat:Cell Phones & Accessories
main_cat:Software
main_cat:Musical Instruments


Аналогично для features_user

In [7]:
user_f = []
col = []
unique_f1 = []
for column in features_user.drop(['userid'], axis=1):
    col += [column]*len(features_user[column].unique())
    unique_f1 += list(features_user[column].unique())
for x,y in zip(col, unique_f1):
    res = str(x)+ ":" +str(y)
    user_f.append(res)
    print(res)

verified:1
verified:0


Вызовем метод fit для нашего датасета

In [8]:
from lightfm.data import Dataset
# we call fit to supply userid, item id and user/item features
dataset1 = Dataset()
dataset1.fit(
        df['userid'].unique(), # all the users
        df['itemid'].unique(), # all the items
        user_features = user_f,
        item_features = item_f
)

Теперь, когда у нас есть готовый скелет датасета, мы готовы добавить в него фактические взаимодействия (interactions) и оценки (ratings).

# Building interactions - построение взаимодействий

Входные данные метода build_interactions - это итерация взаимодействий, где каждое взаимодействие представляет собой кортеж, содержащий три элемента:

     пользователь
     продукт
     вес взаимодействия (опционально)

Вес взаимодействия  означает, что если пользователь «u» взаимодействовал с элементом «i», насколько важно это взаимодействие. С точки зрения нашего примера, вес - это рейтинг, который у нас есть для каждой пары (пользователь, элемент).

In [9]:
# plugging in the interactions and their weights
(interactions, weights) = dataset1.build_interactions([(x[0], x[1], x[2]) for x in df.values ])

Таким образом, матрица взаимодействий сообщает нам, взаимодействовал ли вообще пользователь с элементом, а матрица весов дает количественную оценку этого конкретного взаимодействия.

Мы можем проверить, как выглядят эти две выходные матрицы. Поскольку это разреженные матрицы, мы можем использовать метод .todense (). В обеих матрицах строки - это пользователи, а столбцы - это элементы.

In [10]:
interactions.todense()

matrix([[1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [11]:
weights.todense()

matrix([[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

# Строим Item Features

Метод build_item_features требует ввода в следующем формате:
* [
* (User1, [feature1, feature2, feature3, ....]),
* (User2, [feature1, feature2, feature3, ....]),
* (User3, [feature1, feature2, feature3, ....]),
* .
* .
* ]

     Здесь следует помнить одну очень важную вещь: feature1, feature2, feature3 и т.д. должны быть одним из элементов, присутствующих в списке item_features, который мы передали методу fit в начале.

Вот как сейчас выглядит наш список item_features:
[verified:1, verified:0, unixReviewTime:middle_new, unixReviewTime:new, unixReviewTime:middle_old ... ].


Напишем код, который позволит создать список в требуемом формате:

In [12]:
ll = []
for column in features_item.drop(['itemid'], axis=1):
    ll.append(column + ':')
print(ll)

['main_cat:']


In [13]:
def feature_colon_value(my_list):
    """
    Takes as input a list and prepends the columns names to respective values in the list.
    For example: if my_list = [1,1,0,'del'],
    resultant output = ['f1:1', 'f2:1', 'f3:0', 'loc:del']

    """
    result = []
    aa = my_list
    for x,y in zip(ll,aa):
        res = str(x) +""+ str(y)
        result.append(res)
    return result

In [14]:
ad_subset = features_item.drop(['itemid'], axis=1)
ad_list = [x.tolist() for x in ad_subset.values]
item_feature_list = []
for item in ad_list:
    item_feature_list.append(feature_colon_value(item))
print(f'Final output: {item_feature_list[0:5]}')

Final output: [['main_cat:Grocery'], ['main_cat:Grocery'], ['main_cat:Grocery'], ['main_cat:Grocery'], ['main_cat:Grocery']]


Наконец, мы должны связать каждый элемент item_feature_list с соответствующими идентификаторами продуктов.

In [15]:
item_tuple = list(zip(features_item.itemid, item_feature_list))
item_tuple[0:5]

[(37138, ['main_cat:Grocery']),
 (37138, ['main_cat:Grocery']),
 (37138, ['main_cat:Grocery']),
 (37138, ['main_cat:Grocery']),
 (37138, ['main_cat:Grocery'])]

Мы получили желаемый вид для ввода данных для метода build_item_features. Вызовем этот метод.

In [16]:
item_features = dataset1.build_item_features(item_tuple, normalize= False)
item_features.todense()

matrix([[1., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

В приведенной выше матрице item_features строки - это продукты, а столбцы - это фичи продуктов. 1 присутствует всякий раз, когда у этого продукта есть эта конкретная  фича, присутствующая в тренировочном датасете.

# Создадим user_features

Аналогично item_features

In [17]:
ll = []
for column in features_user.drop(['userid'], axis=1):
    ll.append(column + ':')
print(ll)

['verified:']


In [18]:
ad_subset = features_user.drop(['userid'], axis=1)
ad_list = [x.tolist() for x in ad_subset.values]
user_feature_list = []
for user in ad_list:
    user_feature_list.append(feature_colon_value(user))
print(f'Final output: {user_feature_list[0:5]}')

Final output: [['verified:1'], ['verified:0'], ['verified:1'], ['verified:0'], ['verified:0']]


In [19]:
user_tuple = list(zip(features_user.userid, user_feature_list))

In [20]:
user_features = dataset1.build_user_features(user_tuple, normalize= False)
user_features.todense()

matrix([[1., 0., 0., ..., 0., 6., 0.],
        [0., 1., 0., ..., 0., 4., 1.],
        [0., 0., 1., ..., 0., 6., 1.],
        ...,
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 1., 1., 0.]], dtype=float32)

# Обучаем модель

Создадим словари, с помощью которых по id в датасете LightFM мы сможем находить id в датасете df. 

In [21]:
user_id_map, user_feature_map, item_id_map, item_feature_map = dataset1.mapping()

In [22]:
import scipy.sparse as sparse

from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
import sklearn
from sklearn.model_selection import train_test_split

In [23]:
model = LightFM(loss='warp')
model.fit(interactions, # spase matrix representing whether user u and item i interacted
    user_features = user_features,
    item_features = item_features, # we have built the sparse matrix above
    sample_weight = weights, # spase matrix representing how much value to give to user u and item i inetraction: i.e ratings
    epochs=10)

## Получим значение AUC

In [24]:
# Довольно долго считает, так что закомментируем
# train_auc = auc_score(model,
#                       interactions,
#                       user_features = user_features,
#                       item_features=item_features
#                      ).mean()
# print('Hybrid training set AUC: %s' % train_auc)

## Предсказания

Метод predict принимает три параметра на вход:

*      мэппинги (отображения) id пользователей (например: для получения прогнозов для первого пользователя необходимо передать 0; для второго 1 и т. д.). Эти мэппинги доступны из словаря user_id_map.
*      список id продуктов (опять же не itemid из датасета df, а мэппинги (отображение), которые доступны из item_id_map), для которых вы хотите получить рекомендации.
*      item_features

Предскажем пока на известных данных тренировочного датасета

In [25]:
user_ids = df.userid.apply(lambda x: user_id_map[x])
item_ids = df.itemid.apply(lambda x: item_id_map[x])
preds = model.predict(user_ids.values, item_ids.values, user_features=user_features, item_features=item_features)

In [26]:
sklearn.metrics.roc_auc_score(df.rating,preds)

0.510843338294099

Видим, что добавление фичей негативно сказывается на метрике roc_auc

## Предсказания для новых пользователей/продуктов

Именно поэтому мы в первую очередь создавали гибридную рекомендательную систему.
Для нового пользователя это то, что мы знаем - у него есть значения для feature1, feature2, feature3 как 1,1 и 0 соответственно. Кроме того, verified = 1.

item_feature_list = ['feature: 1', 'feature2: 1', 'feature3: 0', 'verified: 1']

Теперь мы не можем передать это напрямую методу predict. Мы должны преобразовать этот формат в вид, понятный нашей модели lightFM.
В идеале входные данные должны выглядеть как одна из строк в матрице item_features.

Функции ниже преобразует item_feature_list и user_feature_list в требуемый формат.

In [27]:
item_feature_list = ['main_cat:Other']
user_feature_list = ['verified:1']

In [28]:
from scipy import sparse

def format_newitem_input(item_feature_map, item_feature_list): 
    num_features = len(item_feature_list)
    normalised_val = 1.0 
    target_indices = []
    for feature in item_feature_list:
        try:
            target_indices.append(item_feature_map[feature])
        except KeyError:
            print("new item feature encountered '{}'".format(feature))
            pass

    new_item_features = np.zeros(len(item_feature_map.keys()))
    for i in target_indices:
        new_item_features[i] = normalised_val
    new_item_features = sparse.csr_matrix(new_item_features)
    return(new_item_features)

def format_newuser_input(user_feature_map, user_feature_list):
    num_features = len(user_feature_list)
    normalised_val = 1.0 
    target_indices = []
    for feature in user_feature_list:
        try:
            target_indices.append(user_feature_map[feature])
        except KeyError:
            print("new user feature encountered '{}'".format(feature))
            pass

    new_user_features = np.zeros(len(user_feature_map.keys()))
    for i in target_indices:
        new_user_features[i] = normalised_val
    new_user_features = sparse.csr_matrix(new_user_features)
    return(new_user_features)

Наконец, мы можем сделать предсказания для нового пользователя:

In [29]:
new_user_features = format_newuser_input(user_feature_map, user_feature_list)
preds = model.predict(0, item_ids.values, user_features=new_user_features, item_features=item_features)

Здесь первый аргумент, то есть 0, больше не относится к отображаемому идентификатору для первого продукта в датасете. Вместо этого это означает - выберите первую строку разреженной матрицы new_item_features. Передача любого значения, отличного от 0, вызовет ошибку, и это правильно, поскольку в new_item_features нет строк кроме первой строки row0.

Аналогично для нового продукта

In [30]:
new_item_features = format_newitem_input(item_feature_map, item_feature_list)
preds2 = model.predict(user_ids.values, len(user_ids.values)*[0], user_features=user_features, item_features=new_item_features)

И в случае нового пользователя и нового продукта

In [31]:
preds3 = model.predict(0, [0], user_features=new_user_features, item_features=new_item_features)

# Заключение

Таким образом, с помощью добавления item_features и user_features, появляется возможность обойти проблему холодного старта. Мы можем получить информацию о пользователе, например, при регистрации, и использовать эти данные для получения рекомендаций.

### Что можно было бы улучшить?

Заметно высокое потребление памяти. При том, что мы добавляем только по одной фичи для пользователя и продукта, нам едва хватает 16 GB RAM на Kaggle для работы ноутбука. Однокурсником удавалось обойти эту проблему. Возможно, у нас где-то ошибка или выбран не самый оптимальный подход для добавление фичей.